<h1><center><em>Applied Data Science Capstone Project</em></center></h1>

## Introduction

This analysis aims to identify similar neighborhoods in major cities across the United States for the purpose of expanding business franchises. The target audience and stakeholders here are franchise owners, real estate investment firms, and/or banks.

For example, if a pet grooming boutique, vegan bakery, vintage comic book store, fast food, etc. franchise is successful in one neighborhood in NYC and would like to expand to another city, it may be more likely to be successful in a neighborhood with similar characteristics as the original NYC neighborhood.

The franchise is interested in finding the most successful location for their expanding business. The real estate investment firm is interested in buying up lucrative real estate. The bank is interested in understanding the risk/reward of making loan agreements.

## Data

This analysis will rely heavily on the Foursquare location data as well as the neighborhood HTML table data from Wikipedia. Data will be scraped and parsed from Wikipedia using the Python package Beautifulsoup and converted into a pandas dataframe. K-means clustering analysis will be applied to the data. The neighborhoods in cities across the United States will be segmented and clustered into similar neighborhoods.

## Methodology

## Results

## Discussion

## Conclusion